<a href="https://colab.research.google.com/github/sai-darshan-k/SPR/blob/main/Speech_Emotion_Recognition_with_librosa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [15]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import zipfile
import os

# Path to the zip file
zip_path = "/content/drive/MyDrive/Emotion dataset/speech-emotion-recognition-ravdess-data.zip"
# Directory to extract the files
extract_to = "/content/drive/MyDrive/Emotion dataset/"

# Unzipping the file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Files extracted to: {extract_to}")

Files extracted to: /content/drive/MyDrive/Emotion dataset/


In [24]:
import os
Root = "/content/drive/MyDrive/Emotion dataset"
os.chdir(Root)

In [26]:
print("Directory content:", os.listdir(Root))

Directory content: ['speech-emotion-recognition-ravdess-data.zip', 'Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05', 'Actor_06', 'Actor_07', 'Actor_08', 'Actor_09', 'Actor_10', 'Actor_11', 'Actor_12', 'Actor_13', 'Actor_14', 'Actor_15', 'Actor_16', 'Actor_17', 'Actor_18', 'Actor_19', 'Actor_20', 'Actor_21', 'Actor_22', 'Actor_23', 'Actor_24']


In [25]:
ls

Actor_01/  Actor_06/  Actor_11/  Actor_16/  Actor_21/
Actor_02/  Actor_07/  Actor_12/  Actor_17/  Actor_22/
Actor_03/  Actor_08/  Actor_13/  Actor_18/  Actor_23/
Actor_04/  Actor_09/  Actor_14/  Actor_19/  Actor_24/
Actor_05/  Actor_10/  Actor_15/  Actor_20/  speech-emotion-recognition-ravdess-data.zip


In [19]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [27]:
# Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)  # Corrected line
            result = np.hstack((result, mel))
    return result

In [21]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [28]:
# Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob("/content/drive/MyDrive/Emotion dataset/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [29]:
# Split the dataset
x_train, x_test, y_train, y_test = load_data(test_size=0.25)

In [30]:
x_train

array([[-5.22061890e+02,  3.50668907e+01,  3.75342917e+00, ...,
         1.65243138e-04,  1.04321596e-04,  6.55571566e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174108e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05],
       [-6.50705750e+02,  5.30211639e+01, -4.92040396e+00, ...,
         4.75216802e-05,  3.46632551e-05,  1.62844444e-05],
       ...,
       [-5.50096191e+02,  1.70297680e+01, -1.14575634e+01, ...,
         1.51764631e-04,  1.16828531e-04,  8.47479314e-05],
       [-5.55357605e+02,  4.71569710e+01,  1.10750732e+01, ...,
         1.61086471e-04,  1.04962470e-04,  6.52811723e-05],
       [-5.04816345e+02,  3.53618660e+01, -1.43495789e+01, ...,
         6.08151488e-04,  5.55269653e-04,  4.47782222e-04]])

In [31]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(576, 192)


In [32]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [33]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [34]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [35]:
#Predict for the test set
y_pred=model.predict(x_test)

In [36]:
y_pred

array(['happy', 'fearful', 'disgust', 'happy', 'disgust', 'calm', 'calm',
       'disgust', 'disgust', 'happy', 'happy', 'calm', 'fearful', 'happy',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'calm',
       'disgust', 'disgust', 'disgust', 'calm', 'happy', 'happy', 'calm',
       'happy', 'fearful', 'fearful', 'happy', 'disgust', 'happy',
       'fearful', 'happy', 'disgust', 'calm', 'fearful', 'calm',
       'disgust', 'happy', 'disgust', 'calm', 'calm', 'fearful',
       'disgust', 'disgust', 'calm', 'calm', 'happy', 'fearful',
       'fearful', 'disgust', 'happy', 'happy', 'fearful', 'disgust',
       'happy', 'calm', 'calm', 'disgust', 'calm', 'disgust', 'calm',
       'happy', 'calm', 'disgust', 'disgust', 'disgust', 'happy',
       'fearful', 'fearful', 'fearful', 'fearful', 'fearful', 'disgust',
       'fearful', 'happy', 'calm', 'fearful', 'disgust', 'calm',
       'fearful', 'calm', 'disgust', 'disgust', 'disgust', 'fearful',
       'happy', 'fearful', 'disgust',

In [37]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.23%


In [38]:
from sklearn.metrics import accuracy_score, f1_score

In [39]:
f1_score(y_test, y_pred,average=None)

array([0.74074074, 0.64150943, 0.7       , 0.64444444])

In [40]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,happy,happy
1,calm,fearful
2,happy,disgust
3,happy,happy
4,disgust,disgust
5,calm,calm
6,happy,calm
7,happy,disgust
8,disgust,disgust
9,happy,happy


In [41]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.sav', 'wb') as f:
    pickle.dump(model,f)

In [42]:
filename = 'modelForPrediction1.sav'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/drive/MyDrive/Emotion dataset/Actor_01/03-01-01-01-01-01-01.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')